## Baseline + Normalization + Feature Selection

In [184]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from scipy.stats import chi2_contingency

from imblearn.under_sampling import RandomUnderSampler

In [66]:
# Read Normal Datasets
#DATASET
df_path= 'Dataset\hypertension_data.csv'
df = pd.read_csv(df_path)

# mengisi missing value pada kolom sex dengan modus
df['sex'] = df['sex'].fillna(df['sex'].mode()[0])

In [67]:
# # Splitting Data
X = df.drop(columns=['target'])
y = df['target']

# Terapkan Random Under Sampling
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

# Bagi dataset menjadi data latih (training) dan data uji (testing) dengan proporsi 80:20
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [68]:
# Normalisasi Data Train:
X_train_norm = (X_train - X_train.min()) / (X_train.max() - X_train.min())
# Normalisasi Data Test: Gunakan max dan min dari data train
X_test_norm = (X_test - X_train.min()) / (X_train.max() - X_train.min())

In [69]:
df.nunique()

age          88
sex           2
cp            4
trestbps     49
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
target        2
dtype: int64

In [70]:
#Cramer's V Function
def cramers_V(var1,var2) :
  crosstab =np.array(pd.crosstab(var1,var2, rownames=None, colnames=None)) # Cross table building
  stat = chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
  obs = np.sum(crosstab) # Number of observations
  mini = min(crosstab.shape)-1 # Take the minimum value between the columns and the rows of the cross table
  
  return (stat/(obs*mini))

In [236]:
rows= []
for var1 in df:
   cramers =cramers_V(df[var1], df['target']) # Cramer's V test
   rows.append(round(cramers,2)) # Keeping of the rounded value of the Cramer's V
  
cramers_results = np.array(rows)
Cramer_matrix_target = pd.DataFrame(cramers_results[:-1], index =df.columns[:-1], columns=['target'])
threshold = 0.1  # Misalnya, ambang batas korelasi
selected_features = Cramer_matrix_target[Cramer_matrix_target.abs() > threshold].dropna()

print(df.columns, len(df.columns))
print("\nKorelasi dengan Target (Cramer's V):")
print(selected_features)
print('\nJumlah feature yang dipilih:', len(selected_features))

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object') 14

Korelasi dengan Target (Cramer's V):
          target
cp          0.27
trestbps    0.16
chol        0.58
thalach     0.36
exang       0.19
oldpeak     0.30
slope       0.16
ca          0.24
thal        0.29

Jumlah feature yang dipilih: 9


In [238]:
# Assign data train dan data test normalisasi+feature selection

X_train_norm_fs = X_train[selected_features.index]
X_test_norm_fs = X_test[selected_features.index]

X_train_norm_fs

,cp,trestbps,chol,thalach,exang,oldpeak,slope,ca,thal
16520,2,140,185,155,0,3.0,1,0,2
10263,0,130,131,115,1,1.2,1,1,3
24718,0,128,303,159,0,0.0,2,1,2
2801,0,150,247,171,0,1.5,2,0,2
13360,0,146,218,105,0,2.0,1,1,3
...,...,...,...,...,...,...,...,...,...
22248,0,112,149,125,0,1.6,1,0,2
3789,1,101,197,156,0,0.0,2,0,3
11908,2,128,229,150,0,0.4,1,1,3
1954,0,125,300,171,0,0.0,2,2,3


## SVM Model

In [239]:
# Inisialisasi model SVM
from sklearn.svm import SVC
svm_model = SVC(kernel='linear')

In [240]:
# Latih model pada data latih
svm_model.fit(X_train_norm_fs, y_train)

# with open('pickle/svm_model_normalized.pkl', 'wb') as file:
#     pickle.dump(svm_model, file)

SVC(kernel='linear')

In [241]:
# Lakukan prediksi pada data uji
svm_pred = svm_model.predict(X_test_norm_fs)

### 2. Gaussian Naive Bayes

In [262]:
# Inisialisasi model Gaussian Naive Bayes
gnb = GaussianNB()

In [263]:
# Latih model pada data latih
gnb.fit(X_train_norm_fs, y_train)

GaussianNB()

In [264]:
gnb_pred = gnb.predict(X_test_norm_fs)

### 3. Ridge Classifier

In [265]:
ridge_classifier = RidgeClassifier(alpha=1.0, solver='auto', random_state=42)

In [266]:
# Melatih model pada data pelatihan
ridge_classifier.fit(X_train_norm_fs, y_train)

RidgeClassifier(random_state=42)

In [267]:
# Melakukan prediksi pada data uji
rc_pred = ridge_classifier.predict(X_test_norm_fs)

### 4. Linear Discriminant Analysist

In [268]:
lda = LinearDiscriminantAnalysis()

In [269]:
lda.fit(X_train_norm_fs, y_train)

LinearDiscriminantAnalysis()

In [270]:
lda_pred = lda.predict(X_test_norm_fs)

### 5. Logistic Regression

In [271]:
logistic_regression = LogisticRegression(max_iter=1000, random_state=42)

In [272]:
logistic_regression.fit(X_train_norm_fs, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [273]:
lr_pred = logistic_regression.predict(X_test_norm_fs)

## 4. Model evaluator:
1. Precision
2. Recall
3. F1-measure

In [274]:
# Hitung precision, recall, dan F1-score
def eval(y_test,model):
    precision = precision_score(y_test, model, average='weighted')
    recall = recall_score(y_test, model, average='weighted')
    f1 = f1_score(y_test, model, average='weighted')
    print(f'Precision: {precision:.3f}')
    print(f'Recall: {recall:.3f}')
    print(f'F1-score: {f1:.3f}')
    print()
    print(confusion_matrix(y_test, model))
    print(classification_report(y_test, model))
    return

### SVM

In [275]:
eval(y_test, svm_pred)

Precision: 0.842
Recall: 0.840
F1-score: 0.840

[[1896  456]
 [ 299 2073]]
              precision    recall  f1-score   support

           0       0.86      0.81      0.83      2352
           1       0.82      0.87      0.85      2372

    accuracy                           0.84      4724
   macro avg       0.84      0.84      0.84      4724
weighted avg       0.84      0.84      0.84      4724



### 1. Gaussian Naive Bayes

In [276]:
eval(y_test, gnb_pred)

Precision: 0.829
Recall: 0.827
F1-score: 0.827

[[1873  479]
 [ 336 2036]]
              precision    recall  f1-score   support

           0       0.85      0.80      0.82      2352
           1       0.81      0.86      0.83      2372

    accuracy                           0.83      4724
   macro avg       0.83      0.83      0.83      4724
weighted avg       0.83      0.83      0.83      4724



### 3. Ridge Classifier

In [277]:
eval(y_test, rc_pred)

Precision: 0.839
Recall: 0.831
F1-score: 0.830

[[1766  586]
 [ 214 2158]]
              precision    recall  f1-score   support

           0       0.89      0.75      0.82      2352
           1       0.79      0.91      0.84      2372

    accuracy                           0.83      4724
   macro avg       0.84      0.83      0.83      4724
weighted avg       0.84      0.83      0.83      4724



### 4. Linear Discriminant Analysist

In [278]:
eval(y_test, lda_pred)

Precision: 0.839
Recall: 0.831
F1-score: 0.830

[[1766  586]
 [ 214 2158]]
              precision    recall  f1-score   support

           0       0.89      0.75      0.82      2352
           1       0.79      0.91      0.84      2372

    accuracy                           0.83      4724
   macro avg       0.84      0.83      0.83      4724
weighted avg       0.84      0.83      0.83      4724



### 5. Logistic Regression

In [279]:
eval(y_test=y_test, model=lr_pred)

Precision: 0.848
Recall: 0.846
F1-score: 0.846

[[1906  446]
 [ 282 2090]]
              precision    recall  f1-score   support

           0       0.87      0.81      0.84      2352
           1       0.82      0.88      0.85      2372

    accuracy                           0.85      4724
   macro avg       0.85      0.85      0.85      4724
weighted avg       0.85      0.85      0.85      4724

